

This file contains a number of Python oddities, especially for someone with a Fortran/Matlab background. This list is by no means complete, but should give someone beginning with Python an idea what to look out for.

**Zero-based indexing and slices**  
Python uses *zero-based indexing* and when taking a "slice" of a list, the last index *is not* included (first index *is* included):

In [46]:
a = [1,2,3]
print(a[0]) # first element indexed by zero
print(a[0:2]) # upper index not included when taking a slice (lower one is)
print(a[-1]) # last value is indexed by -1, first before last by -2, etc.

1
[1, 2]
3


**Python variables are references**  
In Fortran/Matlab a variable can be seen as a bucket containing an object of a certain type and value. The variable and the object can be interpreted one and the same thing in this approach.

In Python, a variable is always a *reference* to an object (hence Pythonistas use "name" instead of "variable"). The value of the variable is the value of the object it refers to. Objects in Python are either immutable or mutable.

**Immutable objects**  
Objects of the fundamental data-types (int, float, bool and string) are all *immutable*: changing the value of the object is not possible. Assigning a new value to a variable that refers to an immutable object has the consequence that the variable will refer to a new, different, object. This behavior is visible in the following simple example:

In [47]:
a = 1; 
b = a; 
b = 2; 
print('changing b does not change a:',a,b)
c = 2

changing b does not change a: 1 2


Very useful is the id() function (from https://docs.python.org/3/library/functions.html#id): Return the “identity” of an object. This is an integer which is guaranteed to be unique and constant for this object during its lifetime. Two objects with non-overlapping lifetimes may have the same id() value. *CPython implementation detail: This is the address of the object in memory.*

In [48]:
print('id(a) =',id(a))
print('id(b) =',id(b)) # a and b refer to different objects (1 and 2)
print('id(c) =',id(c)) # id(b)==id(c) since they both refer to 2

id(a) = 4420242984
id(b) = 4420243016
id(c) = 4420243016


A *tuple* is a container-like variable (can contain multiple, different, data-types) that is immutable:

In [49]:
a = 1, 2.0 # tuples are created using a comma (parentheses () are optional!)
print(a[0])
#a[0] = 2 # not allowed !

1


**Mutable objects**  
Python also has a other "container-like" data types (list, dict and set). At the deepest level, these will always contain objects of the fundamental (immutable) data-types, but the containers themselves are *mutable*: changing the variable can be interpreted as changing the value of the object to which the variable refers:

In [50]:
lst = [1,2,3]
print(lst,id(lst))
lst[0] = 2
print(lst,id(lst)) # value changed but ID remained the same

[1, 2, 3] 4468478592
[2, 2, 3] 4468478592


An important behavior in python is that assigning a new variable to an existing *mutable* variable only creates an alias. Thus changing the value of the object to which is being referred will be visible in both variables:

In [51]:
lst = [1,2,3]
lst2 = lst
lst[0] = 2
print(lst)
print(id(lst))
print(id(lst2)) # lst and lst2 refer to the same object

[2, 2, 3]
4463295040
4463295040


**Changing arguments of function**  
In programming there are generally two ways of thinking about variables that go into functions: the variables are passed by value, or by reference (this is a huge simplificaiton of a complicated topic).
  
*Passing by value* is easiest interpreted as making a copy of the argument as it enters the function. Modifying the variable therefore does not change the variable outside the function. This is how Matlab functions work, and output variables thus need to be explicitly specified. Note: in practice a copy is only needed if the variable is modified (https://www.mathworks.com/help/matlab/matlab_prog/avoid-unnecessary-copies-of-data.html)
  
*Passing by reference* implies that the variable outside the function is also changed when the variable is modified within the function, since the object that is refered to is changed. In the "variables are buckets" interpretation this is easily understood as changing the variable in the bucket, with the bucket being passed to the function. This change will be visible also outside the function. This is how Fortran subroutines generally work (https://gcc.gnu.org/onlinedocs/gfortran/Argument-passing-conventions.html).
  
Python uses a different approach: *pass by assignment* (https://docs.python.org/3/faq/programming.html#how-do-i-write-a-function-with-output-parameters-call-by-reference).  In this appraoch,
the fact that Python variables are not buckets, but references plays an important role. Also it matters if a variable is mutable or not.  
  
In Python the following rules apply (https://stackoverflow.com/questions/986006/how-do-i-pass-a-variable-by-reference):
- If you pass a mutable object into a method, the method gets a reference to that same object and you can mutate it to your heart's delight, but if you rebind the reference in the method, the outer scope will know nothing about it, and after you're done, the outer reference will still point at the original object.
- If you pass an immutable object to a method, you still can't rebind the outer reference, and you can't even mutate the object.


In [52]:
def myfunc(a):
    a[0] = 2
    return a

a = [1,2,3]
b = myfunc(a) # both a and b are modified by the function

print(a,id(a))
print(b,id(b)) 
        

[2, 2, 3] 4463294272
[2, 2, 3] 4463294272


It is important to note that this only happens when you *modify* the variable. Assigning a completely new value (also called "rebinding" by Pythonistas) to the variable will *not* change the value outside the function (since this will create a new object with a new id)

In [53]:
def myfunc2(a):
    a = [4,5] # rebinding to a new list
    return a

a = [1,2,3]
b = myfunc2(a) # a will not be changed since it is assigned to a new object inside the function

print(a,id(a))
print(b,id(b))

[1, 2, 3] 4462094400
[4, 5] 4463430784


Similarly, assigning a new value to immutable variables results in a new reference to a new object, this will not change the variable outside the function (outside the function, the variable "a" still refers to the original value, because nothing has changed it to refer to anything else)

In [54]:
def print_square(x):
    print(id(x)) # 
    x = x*x # x now refers to a new, different object
    print(id(x))
    print(x)

a = 2
print(id(a))
print_square(a) 
print(a) # the value of a is not changed

4420243016
4420243016
4420243080
4
2


**Changing values of iterator**  
The idea above on arguments in values carries over to Python iterators. In this case, what matters is if the objects *inside* the iterators are mutable or not:

In [55]:
a = [1,2,3] # iterator composed of immutable objects
for i in a:
    i = 4 # this does not change the inital list
print('initial list not changed:',a)

initial list not changed: [1, 2, 3]


In [56]:
a = [[1],[2],[3]] # iterator composed of mutable objects
for i in a:
    i[0] = 4 # this does changes the initial list
print('initial list changed:',a)

initial list changed: [[4], [4], [4]]


Note: similarly to functions, changes to the variable only carry over outside the iteration when you *change* the variable. Assigning a completely new value to the variable will *not* change the value outside the iteration (since this will create a new object with a new id)

In [57]:
a = [[1],[2],[3]] # iterator composed of mutable objects
for i in a:
    i = 4 # this does not changes the initial list
print('initial list not changed:',a)

initial list not changed: [[1], [2], [3]]


**Default values evaluated only once** (from: https://docs.python.org/3/tutorial/controlflow.html#default-argument-values)  
  
Important warning: The default value is evaluated only once. This makes a difference when the default is a mutable object such as a list, dictionary, or instances of most classes. For example, the following function accumulates the arguments passed to it on subsequent calls:

In [58]:
def f(a, L=[]):
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

[1]
[1, 2]
[1, 2, 3]


If you don’t want the default to be shared between subsequent calls, you can write the function like this instead:

In [59]:
def f(a, L=None):
    if L is None:
        L = []
    L.append(a)
    return L

print(f(1))
print(f(2))
print(f(3))

[1]
[2]
[3]


**Truth Value Testing** (from: https://docs.python.org/3/library/stdtypes.html#truth-value-testing)  
  
Any object can be tested for truth value, for use in an if or while condition or as operand of the Boolean operations below.

By default, an object is considered true unless its class defines either a __bool__() method that returns False or a __len__() method that returns zero, when called with the object. 1 Here are most of the built-in objects considered false:

* constants defined to be false: None and False  
* zero of any numeric type: 0, 0.0, 0j, Decimal(0), Fraction(0, 1)  
* empty sequences and collections: '', (), [], {}, set(), range(0)  
* Operations and built-in functions that have a Boolean result always return 0 or False for false and 1 or True for true, unless otherwise stated. (Important exception: the Boolean operations or and and always return one of their operands.)

In [60]:
# taken from: https://stackoverflow.com/questions/1011431/common-pitfalls-in-python

var = 1

if (var == True) :
    # this will execute if var is True or 1, 1.0, 1L
    print('var == True')

if (var != True) :
    # this will execute if var is neither True nor 1
    print('var != True')

if (var == False) :
    # this will execute if var is False or 0 (or 0.0, 0L, 0j)
    print('var == False')

if (var == None) :
    # only execute if var is None
    print('var == Non')

if var :
    # execute if var is a non-empty string/list/dictionary/tuple, non-0, etc
    print('var')

if not var :
    # execute if var is "", {}, [], (), 0, None, etc.
    print('not var')

if var is True :
    # only execute if var is boolean True, not 1
    print('var is True')

if var is False :
    # only execute if var is boolean False, not 0
    print('var is False')

if var is None :
    # same as var == None
    print('var is None')


var == True
var


**"is" vs "=="**  
The symbol "==" is used to compare variables based on their value, and the keword "is" is used to compare the id of objects:

In [61]:
a = [1,2,3]
b = [1,2,3]
print(id(a))
print(id(b))
print(a==b)
print(a is b)

4463418880
4463431616
True
False


For most cases, the "==" symbol is the most suitable. The most important use of the "is" keyword is to check if a has a value of "None" (or a null value, so no value at all), since "==" might be overridden by a class

In [62]:
y = None
print(str(y is None))

True
